In [1]:
OUT_DIR = '/tmp/'
NUM_WORKERS = 4
BATCH_SIZE = 8

from torchvision.models.utils import load_state_dict_from_url

from robustness import model_utils, datasets, train, defaults
from robustness.datasets import Xray
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from robustness import data_augmentation as da
import torch 
import torchvision.datasets
import os
from PIL import ImageFile
from PIL import Image
import torch.nn as nn
    

/home/mugariya.farooq/.conda/envs/mine/lib/python3.7/site-packages/robustness/train.py:24: UserWarning: Could not import amp.
  warnings.warn('Could not import amp.')


In [2]:
# We use cox (http://github.com/MadryLab/cox) to log, store and analyze
# results. Read more at https//cox.readthedocs.io.
from cox.utils import Parameters
import cox.store



In [3]:
normalize = transforms.Normalize(mean=[0.45271412, 0.45271412, 0.45271412],
                                     std=[0.33165374, 0.33165374, 0.33165374])
train_transformer = transforms.Compose([
            transforms.RandomCrop(32, padding=4),
            transforms.RandomHorizontalFlip(),
            transforms.ColorJitter(.25,.25,.25),
            transforms.RandomRotation(2),
            transforms.ToTensor(),
            normalize
])

In [4]:
val_transformer = transforms.Compose([
        transforms.Resize(32),
        transforms.CenterCrop(32),
        transforms.ToTensor(),
        normalize
    ])



batchsize=4
def read_txt(txt_path):
    with open(txt_path) as f:
        lines = f.readlines()
    txt_data = [line.strip() for line in lines]
    return txt_data
class CovidCTDataset(Dataset):
    def __init__(self, root_dir, txt_COVID, txt_NonCOVID, transform=None):

        self.root_dir = root_dir
        self.txt_path = [txt_COVID,txt_NonCOVID]
        self.classes = ['CT_COVID', 'CT_NonCOVID']
        self.num_cls = len(self.classes)
        self.img_list = []
        for c in range(self.num_cls):
            cls_list = [[os.path.join(self.root_dir,self.classes[c],item), c] for item in read_txt(self.txt_path[c])]
            self.img_list += cls_list
        self.transform = transform

    def __len__(self):
        return len(self.img_list)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_path = self.img_list[idx][0]
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)   
        #sample = {'img': image,
               #   'label': int(self.img_list[idx][1])}
        return image,int(self.img_list[idx][1])



In [5]:
trainset = CovidCTDataset(root_dir='/home/mugariya.farooq/Downloads/COVID-CT-master/Images-processed/',
                              txt_COVID='/home/mugariya.farooq/Downloads/COVID-CT-master/Data-split/COVID/trainCT_COVID.txt',
                              txt_NonCOVID='/home/mugariya.farooq/Downloads/COVID-CT-master/Data-split/NonCOVID/trainCT_NonCOVID.txt',
                              transform= train_transformer)
valset = CovidCTDataset(root_dir='/home/mugariya.farooq/Downloads/COVID-CT-master/Images-processed/',
                              txt_COVID='/home/mugariya.farooq/Downloads/COVID-CT-master/Data-split/COVID/valCT_COVID.txt',
                              txt_NonCOVID='/home/mugariya.farooq/Downloads/COVID-CT-master/Data-split/NonCOVID/valCT_NonCOVID.txt',
                              transform= val_transformer)
testset = CovidCTDataset(root_dir='/home/mugariya.farooq/Downloads/COVID-CT-master/Images-processed/',
                              txt_COVID='/home/mugariya.farooq/Downloads/COVID-CT-master/Data-split/COVID/testCT_COVID.txt',
                              txt_NonCOVID='/home/mugariya.farooq/Downloads/COVID-CT-master/Data-split/NonCOVID/testCT_NonCOVID.txt',
                              transform= val_transformer)
                              
                              
print(trainset.__len__())
print(valset.__len__())
#print(testset.__len__())

train_loader = DataLoader(trainset, batch_size=batchsize, drop_last=False, shuffle=True,num_workers=NUM_WORKERS, pin_memory=True)
val_loader = DataLoader(valset, batch_size=batchsize, drop_last=False, shuffle=True,num_workers=NUM_WORKERS, pin_memory=True)
#test_loader = DataLoader(testset, batch_size=batchsize, drop_last=False, shuffle=False)


425
118


In [6]:
ds = Xray('/tmp/train_xray') # FashionMNIST('/tmp')
#dsC= CINIC('/tmp/cinic')
#train_loader = torch.utils.data.DataLoader(ds,batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
#val_loader = torch.utils.data.DataLoader(ds,batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)

#train_loader = train_loader.to(device='cuda')
#val_loader = val_loader.to(device='cuda')

m, _ = model_utils.make_and_restore_model(arch='resnet50', dataset=ds)                          
input = torch.randn((16,3,224,224))
#output = m(input)
#print(output)
#m.fc =   nn.Linear(in_features=2048, out_features=2, bias=True)   
print(m)                
# Create a cox store for logging
out_store = cox.store.Store(OUT_DIR)

# Hard-coded base parameters
train_kwargs = {
    'out_dir': "train_out",
    'adv_train': 1,
    'constraint': '2',
    'eps': 0.5,
    'attack_lr': 0.1,
    'attack_steps': 7,
    'epochs': 150
}
train_args = Parameters(train_kwargs)

# Fill whatever parameters are missing from the defaults
train_args = defaults.check_and_fill_args(train_args,
                        defaults.TRAINING_ARGS, Xray)
train_args = defaults.check_and_fill_args(train_args,
                        defaults.PGD_ARGS, Xray)

AttackerModel(
  (normalizer): InputNormalize()
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (layer1): SequentialWithArgs(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (shortcut): Sequential(
          (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(256, eps=1e-05, mo

In [7]:
# Train a model
train.train_model(train_args, m, (train_loader, val_loader), store=out_store)

Train Epoch:0 | Loss 17.9723 | AdvPrec1 51.765 | AdvPrec5 100.000 | Reg term: 0.
Val Epoch:0 | Loss 0.8735 | NatPrec1 49.153 | NatPrec5 100.000 | Reg term: 0.0 |
Val Epoch:0 | Loss 0.9097 | AdvPrec1 49.153 | AdvPrec5 100.000 | Reg term: 0.0 |
Train Epoch:1 | Loss 1.1198 | AdvPrec1 49.176 | AdvPrec5 100.000 | Reg term: 0.0
Train Epoch:2 | Loss 0.7420 | AdvPrec1 52.706 | AdvPrec5 100.000 | Reg term: 0.0
Train Epoch:3 | Loss 0.7079 | AdvPrec1 54.353 | AdvPrec5 100.000 | Reg term: 0.0
Train Epoch:4 | Loss 0.7198 | AdvPrec1 51.294 | AdvPrec5 100.000 | Reg term: 0.0
Train Epoch:5 | Loss 0.7213 | AdvPrec1 54.588 | AdvPrec5 100.000 | Reg term: 0.0
Val Epoch:5 | Loss 0.7980 | NatPrec1 50.000 | NatPrec5 100.000 | Reg term: 0.0 |
Val Epoch:5 | Loss 0.8027 | AdvPrec1 50.000 | AdvPrec5 100.000 | Reg term: 0.0 |
Train Epoch:6 | Loss 0.7227 | AdvPrec1 52.941 | AdvPrec5 100.000 | Reg term: 0.0
Train Epoch:7 | Loss 0.7222 | AdvPrec1 54.118 | AdvPrec5 100.000 | Reg term: 0.0
Train Epoch:8 | Loss 0.6985 

Train Epoch:71 | Loss 0.6882 | AdvPrec1 53.882 | AdvPrec5 100.000 | Reg term: 0.
Train Epoch:72 | Loss 0.6809 | AdvPrec1 54.353 | AdvPrec5 100.000 | Reg term: 0.
Train Epoch:73 | Loss 0.6870 | AdvPrec1 54.353 | AdvPrec5 100.000 | Reg term: 0.
Train Epoch:74 | Loss 0.6842 | AdvPrec1 54.588 | AdvPrec5 100.000 | Reg term: 0.
Train Epoch:75 | Loss 0.6814 | AdvPrec1 53.882 | AdvPrec5 100.000 | Reg term: 0.
Val Epoch:75 | Loss 0.8033 | NatPrec1 47.458 | NatPrec5 100.000 | Reg term: 0.0 
Val Epoch:75 | Loss 0.8141 | AdvPrec1 47.458 | AdvPrec5 100.000 | Reg term: 0.0 
Train Epoch:76 | Loss 0.6957 | AdvPrec1 52.471 | AdvPrec5 100.000 | Reg term: 0.
Train Epoch:77 | Loss 0.6808 | AdvPrec1 53.176 | AdvPrec5 100.000 | Reg term: 0.
Train Epoch:78 | Loss 0.6880 | AdvPrec1 49.882 | AdvPrec5 100.000 | Reg term: 0.
Train Epoch:79 | Loss 0.6874 | AdvPrec1 51.059 | AdvPrec5 100.000 | Reg term: 0.
Train Epoch:80 | Loss 0.6853 | AdvPrec1 55.059 | AdvPrec5 100.000 | Reg term: 0.
Val Epoch:80 | Loss 0.9310 |

Train Epoch:144 | Loss 0.6849 | AdvPrec1 55.059 | AdvPrec5 100.000 | Reg term: 0
Train Epoch:145 | Loss 0.6841 | AdvPrec1 55.059 | AdvPrec5 100.000 | Reg term: 0
Val Epoch:145 | Loss 2.2555 | NatPrec1 46.610 | NatPrec5 100.000 | Reg term: 0.0
Val Epoch:145 | Loss 2.2959 | AdvPrec1 46.610 | AdvPrec5 100.000 | Reg term: 0.0
Train Epoch:146 | Loss 0.6780 | AdvPrec1 55.059 | AdvPrec5 100.000 | Reg term: 0
Train Epoch:147 | Loss 0.6814 | AdvPrec1 55.059 | AdvPrec5 100.000 | Reg term: 0
Train Epoch:148 | Loss 0.6734 | AdvPrec1 55.059 | AdvPrec5 100.000 | Reg term: 0
Train Epoch:149 | Loss 0.6871 | AdvPrec1 55.059 | AdvPrec5 100.000 | Reg term: 0
Val Epoch:149 | Loss 1.0732 | NatPrec1 50.847 | NatPrec5 100.000 | Reg term: 0.0
Val Epoch:149 | Loss 1.0847 | AdvPrec1 50.847 | AdvPrec5 100.000 | Reg term: 0.0


DataParallel(
  (module): AttackerModel(
    (normalizer): InputNormalize()
    (model): ResNet(
      (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (layer1): SequentialWithArgs(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (shortcut): Sequential(
            (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bia